In [1]:
import shutil
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

import tqdm
try:
    from tqdm.notebook import tqdm
except:
    from tqdm import tqdm

import numpy as np
import os

In [3]:
# Download required files and set the directory
cwd=os.getcwd()
os.chdir(cwd)
cwd+='/'
url = 'https://www.dropbox.com/scl/fi/438cfug8ks6ye05zly4c4/RM_FS_TL.zip?rlkey=iiuadbrbeukf29t1goo7nyt75&st=2v0315ph&dl=1'
destination = cwd+'Files.zip'
import urllib
#Some functions to handle downloads
def download_file(url, destination):
    urllib.request.urlretrieve(url, destination)

def is_download_complete(url, destination):
    # Get the size of the file from the Content-Length header
    with urllib.request.urlopen(url) as response:
        expected_size = int(response.headers['Content-Length'])

    # Get the actual size of the downloaded file
    actual_size = os.path.getsize(destination)

    # Compare the expected size with the actual size
    return expected_size == actual_size

download_file(url, destination)

if is_download_complete(url, destination):
    print("Download complete!")
else:
    print("Download Failed; Please retry")

Download complete!


In [4]:
# Building the file hierachy
''' The file hierachy is as follows

│── RM-FS-TL
│       └── Models
│           └── Blocks_geom*.parquet
│           └── Blocks_geom*.parquet
│           └── Blocks_geom*.parquet
│           └── A_RM__S4_1000.npy (Rot. Machinery dataset "A", i.e., torque 0 Nm; 4-channel dataset)
│           └── A_RM_Classes_1000.npy (Rot. Machinery dataset "A", i.e., torque 0 Nm; number of samples in each class)
│           └── B_RM__S4_1000.npy (Rot. Machinery dataset "B", i.e., torque 2 Nm; 4-channel dataset)
│           └── B_RM_Classes_1000.npy (Rot. Machinery dataset "B", i.e., torque 2 Nm; number of samples in each class)
│           └── C_RM__S4_1000.npy (Rot. Machinery dataset "C", i.e., torque 4 Nm 4-channel dataset)
│           └── C_RM_Classes_1000.npy (Rot. Machinery dataset "C", i.e., torque 4 Nm; number of samples in each class)
│           └── Yellow_S4_1000.npy (Yellow Frame 4-channel dataset)
│           └── Yellow_Classes_1000.npy (Yellow Frame 4-channel dataset; number of samples in each class)
│           └── QUGS_S4_1000.npy (QUGS Frame 4-channel dataset)
│           └── QUGS_Classes_1000.npy (QUGS 4-channel dataset; number of samples in each class)
'''

zip_file_path = cwd+'Files.zip'
extract_dir = cwd

shutil.unpack_archive(zip_file_path, extract_dir)
print(f"Zip file extracted to {extract_dir}")

Zip file extracted to c:\Users\solei\Documents\GitHub\Machine_To_Structure_TL1/


In [5]:
# CL-Network
W=1000
class CustomNeuralNetworkS15(nn.Module):
    def __init__(self):
        super().__init__()
        super(CustomNeuralNetworkS15, self).__init__()
        Stacked_Layers=1
        self.lstm1 = nn.LSTM(int(W/2), 100, Stacked_Layers, batch_first=True)
        self.lstm2 = nn.LSTM(int(W/2), 100, Stacked_Layers, batch_first=True)
        self.lstm3 = nn.LSTM(int(W/2), 100, Stacked_Layers, batch_first=True)
        self.lstm4 = nn.LSTM(int(W/2), 100, Stacked_Layers, batch_first=True)
        self.Flatten = nn.Flatten()
        self.FC1=nn.Linear(400,100)
        self.FC2=nn.Linear(100,1)
        self.LR1 = nn.LeakyReLU(0.2)
        self.LR2 = nn.LeakyReLU(0.2)
        self.LR3 = nn.ReLU()
        self.Sig=nn.Sigmoid()

    def forward(self, x):
        out1,state = self.lstm1(x[:,:,0:int(W/2)])
        out2,state = self.lstm2(x[:,:,int(W/2):2*int(W/2)])
        out3,state = self.lstm3(x[:,:,2*int(W/2):3*int(W/2)])
        out4,state = self.lstm4(x[:,:,3*int(W/2):4*int(W/2)])
        LL=torch.cat((out1,out2,out3,out4),dim=2)
        LLA=self.LR1(LL)
        outCL=self.Flatten(LLA)
        outNF=self.FC1(outCL)
        LLB=self.LR1(outNF)        
        outF=self.FC2(LLB)
        outF=self.LR3(outF)
        return outF
    

In [6]:
'''
As outlined earlier, there exists two folder named PT and NT inside the Models directory. 
The PT folder stores the pre-trained CL models, from which you may reconstruct the paper results. 
Additionally, you may train new CL models which will be stored inside the "NT" folder.
 You may later view results for the new models you trained also.
'''
# Three RM Dataset exists, Torque 0 Nm: 'A', Torque 0 Nm: 'B', Torque 0 Nm: 'C'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
S=4
for RM_N in tqdm(['A','B','C'],desc='for RM datasets A, B, and C'):
    # Rotor data: Class 0: no damage
    Rot_Data=np.load(cwd+'RM_FS_TL/'+RM_N+'_RM_S4_1000.npy')
    # Samples in each Rotor data case:
    Class_N=np.cumsum(np.load(cwd+'RM_FS_TL/'+RM_N+'_RM_Classes_1000.npy'))
    # Minimum is
    print('Min RM class samples:\t',np.min(np.load(cwd+'RM_FS_TL/'+RM_N+'_RM_Classes_1000.npy')))
    # we thus take 
    Class_L=int(np.floor(np.min(np.load(cwd+'RM_FS_TL/'+RM_N+'_RM_Classes_1000.npy'))/100))*100
    print('RM data case length is thus taken as:\t',Class_L)
    # The above process was to make sure we have the same data for the no-damage and damage cases for the training
    
    # Training: As outline in the paper, we only train for the first 9 damage classes
    # RM dataset damage classes
    for tt in tqdm(range(9),desc='RM '+RM_N+' class-by-class training'):
        DD1=Class_L-100
        N=torch.tensor(Rot_Data[0:Class_L-100,:],device='cuda').reshape((Class_L-100,1,2000)).float()
        D=torch.tensor(Rot_Data[int(Class_N[tt]):Class_N[tt]+DD1,:],device='cuda').reshape((DD1,1,2000)).float()

        ## Validation Data
        NV=torch.tensor(Rot_Data[Class_L-100:Class_L,:],device='cuda').reshape((100,1,2000)).float()
        DV=torch.tensor(Rot_Data[Class_N[tt]+DD1:Class_N[tt]+DD1+100,:],device='cuda').reshape((100,1,2000)).float()
        
        # The anchored scores of 0 and 80 in the \theta domain (refer to the article)
        S_1=80
        S_0=0
        Y=torch.tensor(np.zeros((Class_L-100,1)), device='cuda').float()
        Y[:,0]=S_0
        Y1=torch.tensor(np.zeros((Class_L-100,1)), device='cuda').float()
        Y1[:,0]=S_1
    
    
        YV=torch.tensor(np.zeros((100,1)), device='cuda').float()
        YV[:,0]=S_0
        YV1=torch.tensor(np.zeros((100,1)), device='cuda').float()
        YV1[:,0]=S_1
    
        X=torch.cat((N,D),dim=0)
        X_v=torch.cat((NV,DV),dim=0)
        y=torch.cat((Y,Y1),dim=0)
        yv=torch.cat((YV,YV1),dim=0)
        dataset = TensorDataset(X, y)
        data_loader = DataLoader(dataset, 200, shuffle=True)
        
    ########### Training loop
        Net=CustomNeuralNetworkS15().to(device=device)
        Net.train()
        epoch=1000
        crit=torch.nn.functional.mse_loss
        L=[]
        lam=0
        D_solver = torch.optim.Adam(Net.parameters(), lr=5e-4, betas=(0.9, 0.999))
        pr=1
        zet=0
        L_V=[1000]
        L_T=[]
        pp=0
        for i in range(epoch):
            for x_t,y_t in data_loader:
                pred=Net.forward(x_t)
                loss=crit(pred,y_t) 
                D_solver.zero_grad()
                loss.backward()
                D_solver.step()
                if zet==0:
                    L_T.append(loss.item())

                # To automate the whole process in the case of the training loss divergence:
                if loss.item()>max(L_T):
                    Net=CustomNeuralNetworkS15().to(device=device)
                    Net.train()  
                    D_solver = torch.optim.Adam(Net.parameters(), lr=5e-4, betas=(0.9, 0.999))
                    zet=0
                    L_T=[]
                L_T.append(loss.item())

            # Validation loss    
            predA=Net.forward(X_v)
            lossA=crit(predA,yv) 

            # We train for a definite loop and ultimately, the model offering the least
            # validation loss i staken
            if lossA.item()<=np.min(L_V):
                torch.save(Net,cwd+'RM_FS_TL/Models/NT/Model'+RM_N+str(tt+1)+'_W_1000.pth')
                pp=pp+1
            L_V.append(lossA.item())
            zet+=1

for RM datasets A, B, and C:   0%|          | 0/3 [00:00<?, ?it/s]

Min RM class samples:	 1536
RM data case length is thus taken as:	 1500


RM A class-by-class training:   0%|          | 0/9 [00:00<?, ?it/s]

Min RM class samples:	 1536
RM data case length is thus taken as:	 1500


RM B class-by-class training:   0%|          | 0/9 [00:00<?, ?it/s]

Min RM class samples:	 1536
RM data case length is thus taken as:	 1500


RM C class-by-class training:   0%|          | 0/9 [00:00<?, ?it/s]